# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [4]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [5]:
outcome = 'CRIM'
predictors = boston_features.drop('CRIM', axis=1)
pred_sum = "+".join(predictors.columns)
formula = outcome + "~" + pred_sum


In [6]:
model = ols(formula= formula, data=boston_features).fit()
model.summary()

SyntaxError: invalid syntax (<unknown>, line 1)

The above code didn't work so I tried a different version of creating the same OLS model instead below. Not sure why the first didn't work.

In [8]:
import statsmodels.api as sm
predictors_int = sm.add_constant(predictors)
model = sm.OLS(boston_features['CRIM'],predictors_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   CRIM   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     226.7
Date:                Sun, 23 Jun 2019   Prob (F-statistic):          4.75e-185
Time:                        10:03:48   Log-Likelihood:                 489.59
No. Observations:                 506   AIC:                            -955.2
Df Residuals:                     494   BIC:                            -904.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.4030      0.033     12.044      0.000       0.337       0.469
INDUS              0.0456      0.007      6.137      0.000       0.031       0.060
CHAS              -0.0148      0.017     -0.873      0.383      -0.048       0.018
RM                -0.0097      0.009     -1.120      0.263      -0.027       0.007
AGE                0.0200      0.007      2.699      0.007       0.005       0.035
DIS               -0.2467      0.038     -6.510      0.000      -0.321      -0.172
PTRATIO           -0.0133      0.005     -2.610      0.009      -0.023      -0.003
B                 -0.1036      0.020     -5.069      0.000      -0.144      -0.063
LSTAT              0.0167      0.007      2.235      0.026       0.002       0.031
RAD_(0, 6]         0.1005      0.017      5.957      0.000       0.067       0.134
RAD_(6, 24]        0.3025      0.018     16.499      0.000       0.266       0.338
TAX_(0, 270]       0.1050      0.014      7.258      0.000       0.077       0.133
TAX_(270, 360]     0.1367      0.014     10.015      0.000       0.110       0.163
TAX_(360, 712]     0.1613      0.013     12.787      0.000       0.137       0.186
==============================================================================
Omnibus:                        1.816   Durbin-Watson:                   0.566
Prob(Omnibus):                  0.403   Jarque-Bera (JB):                1.652
Skew:                          -0.134   Prob(JB):                        0.438
Kurtosis:                       3.079   Cond. No.                     5.95e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.14e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [9]:
from sklearn.linear_model import LinearRegression
y = boston_features['CRIM']
linreg = LinearRegression()
linreg.fit(predictors, y)
linreg.coef_

array([ 0.04560625, -0.01477087, -0.0097057 ,  0.02003895, -0.24667338,
       -0.01330695, -0.10361591,  0.01673055, -0.10096829,  0.10096829,
       -0.02931793,  0.00233139,  0.02698654])

In [10]:
linreg.intercept_

0.738794001967332

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [11]:
boston_less = boston_features.drop(["RAD_(0, 6]","RAD_(6, 24]", "TAX_(0, 270]", "TAX_(270, 360]", "TAX_(360, 712]"], axis = 1)
boston_less.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114


### Statsmodels

In [13]:
predictors_int2 = sm.add_constant(predictors)
model = sm.OLS(boston_less['CRIM'],predictors_int2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   CRIM   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     226.7
Date:                Sun, 23 Jun 2019   Prob (F-statistic):          4.75e-185
Time:                        10:05:08   Log-Likelihood:                 489.59
No. Observations:                 506   AIC:                            -955.2
Df Residuals:                     494   BIC:                            -904.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.4030      0.033     12.044      0.000       0.337       0.469
INDUS              0.0456      0.007      6.137      0.000       0.031       0.060
CHAS              -0.0148      0.017     -0.873      0.383      -0.048       0.018
RM                -0.0097      0.009     -1.120      0.263      -0.027       0.007
AGE                0.0200      0.007      2.699      0.007       0.005       0.035
DIS               -0.2467      0.038     -6.510      0.000      -0.321      -0.172
PTRATIO           -0.0133      0.005     -2.610      0.009      -0.023      -0.003
B                 -0.1036      0.020     -5.069      0.000      -0.144      -0.063
LSTAT              0.0167      0.007      2.235      0.026       0.002       0.031
RAD_(0, 6]         0.1005      0.017      5.957      0.000       0.067       0.134
RAD_(6, 24]        0.3025      0.018     16.499      0.000       0.266       0.338
TAX_(0, 270]       0.1050      0.014      7.258      0.000       0.077       0.133
TAX_(270, 360]     0.1367      0.014     10.015      0.000       0.110       0.163
TAX_(360, 712]     0.1613      0.013     12.787      0.000       0.137       0.186
==============================================================================
Omnibus:                        1.816   Durbin-Watson:                   0.566
Prob(Omnibus):                  0.403   Jarque-Bera (JB):                1.652
Skew:                          -0.134   Prob(JB):                        0.438
Kurtosis:                       3.079   Cond. No.                     5.95e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.14e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Scikit-learn

In [14]:
y = boston_less['CRIM']
linreg = LinearRegression()
linreg.fit(predictors, y)
linreg.coef_

array([ 0.04560625, -0.01477087, -0.0097057 ,  0.02003895, -0.24667338,
       -0.01330695, -0.10361591,  0.01673055, -0.10096829,  0.10096829,
       -0.02931793,  0.00233139,  0.02698654])

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.